In [ ]:
import sys
from random import random
from operator import add

In [ ]:
from pyspark.sql import SparkSession
from bigdl.ppml.utils.ppml_conf import PPMLConf

In [ ]:
def square_2_random(_: int) -> float:
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

In [ ]:
ppml_conf = (PPMLConf(sgx_enabled = True)
        .setAppName("sgx-pyspark-pi-notebook-example")
        .set("spark.driver.memory", "32g")
        .set("spark.executor.cores", "8")
        .set("spark.executor.memory", "32g")
        .set("spark.executor.instances", "2")
        .set("spark.cores.max", "32"))

In [ ]:
if __name__ == '__main__':
    sc = SparkSession.builder.config(conf=(ppml_conf.conf())).getOrCreate()
    partiton_num = 2
    n = 100000 * partiton_num
    count = sc.sparkContext.parallelize(range(1, n + 1), partiton_num).map(square_2_random).reduce(add)
    sc.stop()

In [ ]:
print("Pi is roughly %f" % (4.0 * count / n))